In [20]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.1 MB/s eta 0:00:00a 0:00:01


In [2]:
from pypdf import PdfReader

In [3]:
reader = PdfReader("microsoft_annual_report_2022.pdf")

In [4]:
pdf_texts = [ p.extract_text().strip() for p in reader.pages]

In [5]:
len(pdf_texts)

93

In [15]:
# pdf_texts[3]

In [13]:
# def word_wrap(string, n_chars=72):
#     # Wrap a string at the next space after n_chars
#     if len(string) < n_chars:
#         return string
#     else:
#         return string[:n_chars].rsplit(' ', 1)[0] + '\n' + word_wrap(string[len(string[:n_chars].rsplit(' ', 1)[0])+1:], n_chars)
# word_wrap(pdf_texts[3])

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter


In [16]:
character_splitter =  RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)

character_split_text = character_splitter.split_text('\n\n'.join(pdf_texts))

In [18]:
# character_split_text

In [22]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_text:
    token_split_texts += token_splitter.split_text(text)

In [23]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction


In [26]:
emb_func = SentenceTransformerEmbeddingFunction()
chromadb_client = chromadb.Client()
chroma_collection = chromadb_client.create_collection("microsoft_annual_report_2022",embedding_function=emb_func)

In [28]:
ids = [ str(i) for i in range(len(token_split_texts))]

In [30]:
chroma_collection.add(ids=ids,documents=token_split_texts)

In [41]:
query="what is the total revenue"
rel_docs = chroma_collection.query(query_texts=[query],n_results=5)

In [1]:
import os
import openai
from openai import OpenAI

%set_env OPENAI_API_KEY = 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

env: OPENAI_API_KEY=


In [43]:
information = "\n".join(rel_docs)
messages = [
    {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report."
            "You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
    },
    {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
]

In [44]:
response =  openai_client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = messages
)


In [45]:
response

ChatCompletion(id='chatcmpl-9K6GyOkNNPvFatoC3GTi4pcgSbX4I', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm sorry, but the information provided does not include anything related to total revenue. Could you please provide more details or specify if you are looking for something else from the annual report?", role='assistant', function_call=None, tool_calls=None))], created=1714577700, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3b956da36b', usage=CompletionUsage(completion_tokens=37, prompt_tokens=89, total_tokens=126))